# 04 Process In Data

In [2]:
# 04 process input data

import time
import datetime
import medareda_lib
import process

In [3]:
process.processSinglePrice()


Rows: 



In [ ]:
while True:
    #time.sleep(3)
    process.processSinglePrice()
    
print 'Done.'




Rows: 


Rows: 


Rows: 


Rows: 
